In [61]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import json
import csv

In [55]:
header = ['URL','Name', 'Brand','Model','itemCondition','modelDate','manufacturer','fuelType',
          'vehicleTransmission','color','Engine_Displacement(cc)','mileageFromOdometer (km)',
          'Offering_Price','priceCurrency','car_specs']
with open("lhr_data.csv",'w',newline='',encoding='UTF8') as f:
    writer= csv.writer(f)
    writer.writerow(header)

In [62]:
def scrape_car_urls(url):
    car_data=[]
    
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    
    results = soup.find("div", class_="used-car-search-results")
    if results is None:
        print("Error: Unable to find results.")
        return None
    
    car_elements = results.find("div", class_="col-md-9 search-listing pull-right")
    if car_elements is None:
        print("Error: Unable to find car elements.")
        return None
    hrefs = car_elements.find_all("a", {"class": "car-name ad-detail-path"})
    
    for href in hrefs:
        car_url = href.get('href')
        car_data.append(car_url)
        with open("lhr_data.csv",'a+',newline='',encoding='UTF8') as f:
            writer= csv.writer(f)
            writer.writerow([car_url])
    
    next_page = soup.find("li", class_="next_page")
    if next_page:
        next_page_url = next_page.a.get('href')
        car_data += scrape_car_urls("https://www.pakwheels.com" + next_page_url)
        
    
    return car_data



In [63]:
# Initial function call
initial_url = "https://www.pakwheels.com/used-cars/lahore/24858"
car_data = scrape_car_urls(initial_url)



In [64]:
header = ['URL','Name', 'Brand','Model','itemCondition','modelDate','manufacturer','fuelType',
          'vehicleTransmission','color','Engine_Displacement(cc)','mileageFromOdometer (km)',
          'Offering_Price','priceCurrency','car_specs']


with open("lhr_car_data.csv",'w',newline='',encoding='UTF8') as f:
    writer= csv.writer(f)
    writer.writerow(header)

In [65]:
def scrape_car_data(url):
    if not url.startswith("http"):
        url = "http://www.pakwheels.com" + url
    page = requests.get(url)
    soup1 = BeautifulSoup(page.text, "html.parser")
    results = soup1.find(id="main-container")
    
    if results is None:
        print("Error: Unable to find the main container.")
        return None
    
    car_elements = results.find("div", class_="col-md-8")
    
    if car_elements is None:
        print("Error: Unable to find car elements.")
        return None
    
    car_info = car_elements.script.text
    
    try:
        car_object = json.loads(car_info)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        print(url)
        return None
   
    car_specs = results.find("ul", class_="list-unstyled car-feature-list nomargin")
    if car_specs:
        car_specs= car_specs.text
        car_spec_features=car_specs.replace("\n",",")[2:]
    else:
        car_spec_features=False
    
    car_name = car_object["name"]
    car_brand = car_object["brand"]["name"]
    car_Model = car_object["model"]
    car_itemCondition = car_object["itemCondition"]
    car_modelDate = car_object["modelDate"]
    car_manufacturer = car_object["manufacturer"]
    car_fuelType = car_object["fuelType"]
    car_vehicleTransmission = car_object["vehicleTransmission"]
    car_color = car_object["color"]
    car_Engine_Displacement = car_object["vehicleEngine"]["engineDisplacement"]
    car_mileageFromOdometer = car_object["mileageFromOdometer"]
    car_Offering_Price = car_object["offers"]["price"]
    car_priceCurrency = car_object["offers"]["priceCurrency"]
    car_mileageFromOdometer = ''.join(filter(str.isdigit, car_mileageFromOdometer))
    car_Engine_Displacement = int(re.search(r'\d+', car_Engine_Displacement).group()) if re.search(r'\d+', car_Engine_Displacement) else None
    
    car_info = [url,car_name, car_brand, car_Model, car_itemCondition, car_modelDate, car_manufacturer, car_fuelType,
                car_vehicleTransmission, car_color, car_Engine_Displacement, car_mileageFromOdometer, car_Offering_Price,
                car_priceCurrency, car_spec_features]
    
    with open("lhr_car_data.csv",'a+',newline='',encoding='UTF8') as f:
            writer= csv.writer(f)
            writer.writerow(car_info)




In [66]:
for Url in car_data:
    scrape_car_data(Url)

Error decoding JSON: Expecting value: line 2 column 5 (char 5)
http://www.pakwheels.com/used-cars/toyota-fortuner-2022-for-sale-in-lahore-7438055
Error decoding JSON: Expecting value: line 2 column 5 (char 5)
http://www.pakwheels.com/used-cars/toyota-hilux-2021-for-sale-in-lahore-7479274
Error decoding JSON: Expecting value: line 2 column 5 (char 5)
http://www.pakwheels.com/used-cars/toyota-corolla-2022-for-sale-in-lahore-7334845
Error decoding JSON: Expecting value: line 2 column 5 (char 5)
http://www.pakwheels.com/used-cars/toyota-corolla-2019-for-sale-in-lahore-7485466
Error decoding JSON: Expecting value: line 2 column 5 (char 5)
http://www.pakwheels.com/used-cars/toyota-fortuner-2022-for-sale-in-lahore-7438055
Error decoding JSON: Expecting value: line 2 column 5 (char 5)
http://www.pakwheels.com/used-cars/toyota-hilux-2021-for-sale-in-lahore-7479274
Error decoding JSON: Expecting value: line 2 column 5 (char 5)
http://www.pakwheels.com/used-cars/toyota-corolla-2022-for-sale-in-la

Error decoding JSON: Expecting value: line 2 column 5 (char 5)
http://www.pakwheels.com/used-cars/honda-city-2022-for-sale-in-lahore-7476909
Error decoding JSON: Expecting value: line 2 column 5 (char 5)
http://www.pakwheels.com/used-cars/honda-freed-2015-for-sale-in-lahore-7476907
Error decoding JSON: Expecting value: line 2 column 5 (char 5)
http://www.pakwheels.com/used-cars/bmw-x1-2017-for-sale-in-lahore-7476903
Error decoding JSON: Expecting value: line 2 column 5 (char 5)
http://www.pakwheels.com/used-cars/honda-freed-2013-for-sale-in-lahore-7476901
Error decoding JSON: Expecting value: line 2 column 5 (char 5)
http://www.pakwheels.com/used-cars/mercedes-benz-eqs-2023-for-sale-in-lahore-7342184
Error decoding JSON: Expecting value: line 2 column 5 (char 5)
http://www.pakwheels.com/used-cars/suzuki-alto-2022-for-sale-in-lahore-7341884
Error decoding JSON: Expecting value: line 2 column 5 (char 5)
http://www.pakwheels.com/used-cars/toyota-hilux-2022-for-sale-in-lahore-7474737
Error

Error decoding JSON: Expecting value: line 2 column 5 (char 5)
http://www.pakwheels.com/used-cars/honda-city-2022-for-sale-in-lahore-7476909
Error decoding JSON: Expecting value: line 2 column 5 (char 5)
http://www.pakwheels.com/used-cars/honda-freed-2015-for-sale-in-lahore-7476907
Error decoding JSON: Expecting value: line 2 column 5 (char 5)
http://www.pakwheels.com/used-cars/bmw-x1-2017-for-sale-in-lahore-7476903
Error decoding JSON: Expecting value: line 2 column 5 (char 5)
http://www.pakwheels.com/used-cars/honda-freed-2013-for-sale-in-lahore-7476901
Error decoding JSON: Expecting value: line 2 column 5 (char 5)
http://www.pakwheels.com/used-cars/mercedes-benz-eqs-2023-for-sale-in-lahore-7342184
Error decoding JSON: Expecting value: line 2 column 5 (char 5)
http://www.pakwheels.com/used-cars/toyota-prius-2020-for-sale-in-lahore-7443525
Error decoding JSON: Expecting value: line 2 column 5 (char 5)
http://www.pakwheels.com/used-cars/honda-vezel-2022-for-sale-in-lahore-7443636
Error

Error decoding JSON: Expecting value: line 2 column 5 (char 5)
http://www.pakwheels.com/used-cars/suzuki-wagon-r-2019-for-sale-in-lahore-7463111
Error decoding JSON: Expecting value: line 2 column 5 (char 5)
http://www.pakwheels.com/used-cars/toyota-prado-2017-for-sale-in-lahore-7448224
Error decoding JSON: Expecting value: line 2 column 5 (char 5)
http://www.pakwheels.com/used-cars/toyota-hilux-2022-for-sale-in-lahore-7385062
Error decoding JSON: Expecting value: line 2 column 5 (char 5)
http://www.pakwheels.com/used-cars/toyota-prius-2020-for-sale-in-lahore-7356301
Error decoding JSON: Expecting value: line 2 column 5 (char 5)
http://www.pakwheels.com/used-cars/honda-grace-hybrid-2019-for-sale-in-lahore-7103767
Error decoding JSON: Expecting value: line 2 column 5 (char 5)
http://www.pakwheels.com/used-cars/toyota-passo-2018-for-sale-in-lahore-6110636
Error decoding JSON: Expecting value: line 2 column 5 (char 5)
http://www.pakwheels.com/used-cars/toyota-yaris-2020-for-sale-in-lahore

Error decoding JSON: Expecting value: line 2 column 5 (char 5)
http://www.pakwheels.com/used-cars/subaru-stella-2007-for-sale-in-lahore-7003850
Error decoding JSON: Expecting value: line 2 column 5 (char 5)
http://www.pakwheels.com/used-cars/suzuki-spacia-2018-for-sale-in-lahore-6110631
Error decoding JSON: Expecting value: line 2 column 5 (char 5)
http://www.pakwheels.com/used-cars/nissan-note-2017-for-sale-in-lahore-6700968
Error decoding JSON: Expecting value: line 2 column 5 (char 5)
http://www.pakwheels.com/used-cars/toyota-prado-2011-for-sale-in-lahore-7387635
Error decoding JSON: Expecting value: line 2 column 5 (char 5)
http://www.pakwheels.com/used-cars/toyota-aqua-2018-for-sale-in-lahore-7242477
Error decoding JSON: Expecting value: line 2 column 5 (char 5)
http://www.pakwheels.com/used-cars/honda-city-2023-for-sale-in-lahore-7126216
Error decoding JSON: Expecting value: line 2 column 5 (char 5)
http://www.pakwheels.com/used-cars/suzuki-wagon-r-2020-for-sale-in-lahore-7377958